In [1]:
!pip install streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 125.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 138.0 MB/s eta 0:00:00


In [2]:
# 1. Instal pustaka utama
!pip install streamlit pyngrok transformers accelerate bitsandbytes diffusers evaluate rouge_score pandas openpyxl pillow scipy scikit-learn

# 2. Instal FAISS (Versi CPU, paling mudah dan stabil untuk deployment cepat)
!pip install faiss-cpu

# PENTING: Jalankan kembali semua sel instalasi/setup Anda di Colab.

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.9 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=7955bc62bb3402d0b2aafbb8d7bfebdf043845b0294eab7465c5a137beb2c42e
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 45.1 MB/s eta 0:00:00


In [3]:
# ================= DOWNLOAD ZIP =================
!wget -O streamlit_assets.zip \
https://github.com/Gammaa12/batik-nitik-generative-ai/raw/main/streamlit_assets.zip

# ================= EXTRACT =================
!unzip -q streamlit_assets.zip -d /content

# ================= CEK ISI =================
import os
print(os.listdir('/content'))

--2025-12-29 00:19:36--  https://github.com/Gammaa12/batik-nitik-generative-ai/raw/main/streamlit_assets.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/Gammaa12/batik-nitik-generative-ai/main/streamlit_assets.zip [following]
--2025-12-29 00:19:36--  https://media.githubusercontent.com/media/Gammaa12/batik-nitik-generative-ai/main/streamlit_assets.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174636359 (167M) [application/zip]
Saving to: ‘streamlit_assets.zip’

streamlit_assets.zi 100%[===================>] 166.55M  16.7MB/s    in 10s     

2025-12-29 00:19:47 (16.7

In [4]:
# Pindahkan folder streamit_assets ke lokasi yang benar (/content/)
!mv /content/content/streamit_assets /content/

# Hapus folder 'content' yang sekarang kosong yang menyebabkan masalah
!rm -rf /content/content/

# Verifikasi ulang jalur
!ls /content/streamit_assets/outputs/faiss.index

print("\n✅ Jalur file FAISS Index sekarang sudah diperbaiki!")
print("Sekarang file dapat diakses oleh Streamlit di jalur relatif: streamit_assets/outputs/faiss.index")

/content/streamit_assets/outputs/faiss.index

✅ Jalur file FAISS Index sekarang sudah diperbaiki!
Sekarang file dapat diakses oleh Streamlit di jalur relatif: streamit_assets/outputs/faiss.index


In [5]:
%%writefile app.py

import streamlit as st
import torch
import numpy as np
import pandas as pd
import os
import faiss
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, Blip2Processor, Blip2ForConditionalGeneration
from diffusers import StableDiffusionPipeline
from transformers import BitsAndBytesConfig

# ===============================================
# 0. KONFIGURASI PATHS
# ===============================================
CONFIG = {
    "model_name": "Salesforce/blip2-opt-2.7b",
    "output_model_dir": "streamit_assets/blip2_batik_qlora_v2",
    "metadata_path": "streamit_assets/Batik Nitik 960 Metadata - Indonesian English v3.xlsx",
    "clip_outputs_dir": "streamit_assets/outputs",
    "preprocessed_sd_dir": "streamit_assets/sd_images",
}
device = "cuda" if torch.cuda.is_available() else "cpu"

# ===============================================
# 1. PEMUATAN MODEL DENGAN CACHE
# ===============================================

@st.cache_resource
def load_models_and_index():
    st.write("⏳ Memuat Model & Data (Multimodal CLIP Ready)...")
    index = faiss.read_index(os.path.join(CONFIG["clip_outputs_dir"], "faiss.index"))
    meta = np.load(os.path.join(CONFIG["clip_outputs_dir"], "clip_metadata.npy"), allow_pickle=True)
    df_raw = pd.read_excel(CONFIG["metadata_path"])
    df_raw.columns = df_raw.columns.str.strip()
    metadata_lookup = {
        str(k).strip(): str(v).strip()
        for k, v in zip(df_raw['Batik Nitik Motif'], df_raw['Description'])
    }
    clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    processor = Blip2Processor.from_pretrained(CONFIG["output_model_dir"])
    model = Blip2ForConditionalGeneration.from_pretrained(
        CONFIG["model_name"],
        quantization_config=bnb,
        device_map="auto"
    )
    model.load_adapter(CONFIG["output_model_dir"])
    model.eval()

    sd_pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16,
        safety_checker=None
    ).to(device)

    return clip_model, clip_processor, processor, model, sd_pipe, index, meta, metadata_lookup

# ===============================================
# 2. FUNGSI UTILITY
# ===============================================

def get_motif_name_from_filename(filename):
    base_name = os.path.basename(filename).split('.')[0]
    parts = base_name.split(' ')
    try:
        if parts[0].replace('.', '', 1).isdigit(): parts = parts[1:]
        if '_' in parts[-1] or parts[-1].isdigit(): parts = parts[:-1]
        motif_name = ' '.join(parts).strip()
        final_motif = ' '.join([word for word in motif_name.split() if not word.replace('.', '', 1).isdigit()]).strip()
        return final_motif if final_motif else "Unknown Motif"
    except: return "Unknown Motif"

def embed_image_clip(img_path, clip_processor, clip_model):
    img = Image.open(img_path).convert("RGB")
    inputs = clip_processor(images=img, return_tensors="pt").to(device)
    with torch.no_grad():
        emb = clip_model.get_image_features(**inputs)
        emb /= emb.norm(dim=-1, keepdim=True)
    return emb.cpu().numpy().flatten()

def embed_text_clip(text_query, clip_processor, clip_model):
    inputs = clip_processor(text=[text_query], return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        emb = clip_model.get_text_features(**inputs)
        emb /= emb.norm(dim=-1, keepdim=True)
    return emb.cpu().numpy().flatten()

def generate_visual_description(img_path, processor, model):
    img = Image.open(img_path).convert("RGB")
    prompt_visual = "Question: Describe the pattern and color of this batik. Answer:"
    inputs = processor(images=img, text=prompt_visual, return_tensors="pt").to(device)
    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=inputs.input_ids,
            pixel_values=inputs.pixel_values,
            max_new_tokens=50,
            do_sample=False,
            num_beams=3,
            repetition_penalty=1.5
        )
    caption_raw = processor.decode(generated_ids[0], skip_special_tokens=True).strip()
    if "Answer:" in caption_raw:
        caption_clean = caption_raw.split("Answer:")[-1].strip()
    else:
        caption_clean = caption_raw.replace("Question: Describe the pattern and color of this batik. Answer:", "").strip()
    return caption_clean if len(caption_clean) > 5 else "A detailed geometric batik pattern"

# ===============================================
# 3. INTERFACE STREAMLIT
# ===============================================

st.set_page_config(page_title="Batik Nitik Multimodal RAG", layout="wide")
st.title("🧵 Batik Nitik RAG Generatif Hibrida")

try:
    CLIP_M, CLIP_P, BLIP_P, BLIP_M, SD_PIPE, INDEX, META, METADATA_LOOKUP = load_models_and_index()
    st.success("✅ Sistem Multimodal Siap.")
except Exception as e:
    st.error(f"❌ Error: {e}"); st.stop()

with st.sidebar:
    st.header("🔍 Input Query")
    uploaded_file = st.file_uploader("Opsi 1: Unggah Gambar", type=["jpg", "jpeg"])
    text_input = st.text_input("Opsi 2: Masukkan Teks (Contoh: 'Batik warna coklat bunga')")

    st.header("⚙️ Parameter")
    top_k = st.slider("Jumlah Retrieval", 1, 10, 5)
    strength = st.slider("Strength (Hanya jika ada gambar)", 0.1, 1.0, 0.55)
    guidance = st.slider("Guidance Scale", 5.0, 15.0, 7.5)

# Tentukan Logika Query
query_vector = None
query_label = "Custom Batik"
base_image = None

if uploaded_file is not None:
    temp_query_path = os.path.join("/tmp", uploaded_file.name)
    with open(temp_query_path, "wb") as f: f.write(uploaded_file.getbuffer())
    query_vector = embed_image_clip(temp_query_path, CLIP_P, CLIP_M)
    query_label = get_motif_name_from_filename(uploaded_file.name)
    base_image = Image.open(temp_query_path).convert("RGB").resize((512,512))
    st.sidebar.image(base_image, caption="Query Gambar digunakan.")
elif text_input:
    query_vector = embed_text_clip(text_input, CLIP_P, CLIP_M)
    query_label = text_input
    st.sidebar.info(f"Query Teks: '{text_input}' digunakan.")

# EKSEKUSI
if query_vector is not None:
    # 1. Tampilkan Input Query Terlebih Dahulu
    st.subheader("🖼️ Query Input")
    if base_image: st.image(base_image, width=300)
    else: st.info(f"Mode Pencarian Teks: **{text_input}**")

    if st.button("🚀 Jalankan Proses", use_container_width=True):
        # 2. PROSES RETRIEVAL
        st.write("---")
        st.subheader("🎯 1. Hasil Retrieval Terkait")

        # Search FAISS
        D, I = INDEX.search(query_vector.reshape(1, -1).astype("float32"), top_k)

        retrieved_data = []
        fused_elements = []

        # Tampilkan Hasil Gambar Retrieval
        cols = st.columns(top_k if top_k <= 5 else 5)
        for i, (dist, idx) in enumerate(zip(D[0], I[0])):
            filename = META[idx]
            # Hitung Cosine Similarity (Sistem FAISS L2 ke Cosine approx)
            similarity_score = max(0, min(100, (1 - dist/2) * 100))
            img_path_sd = os.path.join(CONFIG["preprocessed_sd_dir"], filename)
            motif_name = get_motif_name_from_filename(filename).strip()

            # Tampilkan di kolom Streamlit
            with cols[i % 5]:
                st.image(img_path_sd, use_container_width=True)
                st.caption(f"**File:** {filename}")
                st.caption(f"**Motif:** {motif_name}")
                st.caption(f"**Sim:** {similarity_score:.2f}%")

            # Simpan data untuk proses Fusion di tahap berikutnya
            retrieved_data.append({
                "filename": filename,
                "motif": motif_name,
                "path": img_path_sd
            })

        # 3. PROSES GENERATE CAPTION & FUSION (RAG)
        st.write("---")
        st.subheader("📝 2. Analisis Visual & RAG Fusion")

        with st.spinner("Menganalisis gaya visual dari gambar referensi..."):
            for item in retrieved_data:
                factual_metadata = METADATA_LOOKUP.get(item['motif'], "Traditional Indonesian batik motif.")
                # Memanggil BLIP-2
                visual_desc = generate_visual_description(item['path'], BLIP_P, BLIP_M)

                element = f"[Motif: {item['motif']} | Factual: {factual_metadata[:100]} | Style: {visual_desc}]"
                fused_elements.append(element)

            full_rich_context = " ".join(fused_elements)
            final_prompt = (
                f"A professional photo of {query_label} batik fabric, symmetrical, "
                f"intricate detail, high resolution. Inspired by: {full_rich_context}"
            )

            with st.container(border=True):
                st.write("**Final Fusion Prompt (untuk Stable Diffusion):**")
                st.code(final_prompt, wrap_lines=True)

        # 4. GENERASI GAMBAR AKHIR
        st.write("---")
        st.subheader("🌟 3. Hasil Generasi Akhir")
        with st.spinner("🎨 Menghasilkan desain batik baru..."):
            if base_image:
                output = SD_PIPE(prompt=final_prompt, image=base_image, strength=strength, guidance_scale=guidance).images[0]
            else:
                output = SD_PIPE(prompt=final_prompt, guidance_scale=guidance).images[0]

            st.image(output, use_container_width=True, caption=f"Desain Baru Terinspirasi dari {query_label}")

else:
    st.info("Pilih input di sidebar untuk memulai proses.")

Writing app.py


In [6]:
# GANTI DENGAN AUTH TOKEN NGROK ANDA!
NGROK_AUTH_TOKEN = "36shVWptNnk5Ecl4XiUAErRPaEN_4VfKLR6PaKLr11c7mbiv5"

from pyngrok import ngrok
import subprocess

# Set Auth Token ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Jalankan Streamlit di background (port 8501)
print("Starting Streamlit in background...")
proc = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.enableCORS", "false"])

# Buat tunnel ngrok
print("Starting ngrok tunnel...")
public_url = ngrok.connect(8501).public_url

print(f"=============================================================")
print(f"🚀 Streamlit Public URL (Akses Presentasi Anda):")
print(f"   {public_url}")
print(f"=============================================================")
print("JANGAN TUTUP NOTEBOOK INI! Sesi akan mati setelah 1-2 jam atau jika idle.")

Starting Streamlit in background...
Starting ngrok tunnel...
🚀 Streamlit Public URL (Akses Presentasi Anda):
   https://unappealably-undawning-susanna.ngrok-free.dev
JANGAN TUTUP NOTEBOOK INI! Sesi akan mati setelah 1-2 jam atau jika idle.
